## import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

## cuda 확인, cuda 환경, cpu환경 둘 다 사용가능하게 하기

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

## 여러개의 배치로 잘라 데이터를 처리하기 with 좌우를 뒤바꿔 학습 데이터 늘리기

In [5]:
EPOCHS = 50
BATCH_SIZE = 64

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data/',
                  train=True,
                  download=True,
                  transform=transforms.Compose([
                      transforms.RandomHorizontalFlip(),
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081,))
                  ])),
    batch_size=BATCH_SIZE, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data/',
                  train=False,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081,))
                  ])),
    batch_size=BATCH_SIZE, shuffle=True
)

100%|██████████████████████████████████████████████████████████████████| 9912422/9912422 [00:00<00:00, 13699500.94it/s]


Extracting ./.data/MNIST\raw\train-images-idx3-ubyte.gz to ./.data/MNIST\raw



100%|███████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 4125032.14it/s]


Extracting ./.data/MNIST\raw\train-labels-idx1-ubyte.gz to ./.data/MNIST\raw



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 4444386.08it/s]


Extracting ./.data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./.data/MNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./.data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./.data/MNIST\raw



 ## 이미지 분류를 위한 인공 신경망 구현 with 드롭아웃

In [6]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout_p = dropout_p

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = self.fc3(x)
        return x

## cuda일 경우, gpu 아닐 경우, cpu로 보내준다

In [7]:
model = Net(dropout_p=0.2).to(DEVICE)

## 최적화 알고리즘 -> optim.SGD = 최저화를 위한 화률적 경사하강법

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

## 학습 함수, 모델의 가중치를 조절하는 작업

In [9]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

## 모델의 일반화를 평가하고, 학습을 언제 멈춰야 할지 판단하기

In [10]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 학습

In [11]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss: 0.5486, Accuracy: 82.28%
[2] Test Loss: 0.4174, Accuracy: 87.10%
[3] Test Loss: 0.3416, Accuracy: 89.19%
[4] Test Loss: 0.2888, Accuracy: 91.22%
[5] Test Loss: 0.2536, Accuracy: 92.10%
[6] Test Loss: 0.2324, Accuracy: 92.99%
[7] Test Loss: 0.2138, Accuracy: 93.49%
[8] Test Loss: 0.1973, Accuracy: 93.96%
[9] Test Loss: 0.1869, Accuracy: 94.31%
[10] Test Loss: 0.1738, Accuracy: 94.69%
[11] Test Loss: 0.1646, Accuracy: 94.99%
[12] Test Loss: 0.1542, Accuracy: 95.25%
[13] Test Loss: 0.1485, Accuracy: 95.42%
[14] Test Loss: 0.1431, Accuracy: 95.48%
[15] Test Loss: 0.1407, Accuracy: 95.66%
[16] Test Loss: 0.1364, Accuracy: 95.70%
[17] Test Loss: 0.1311, Accuracy: 96.00%
[18] Test Loss: 0.1260, Accuracy: 96.01%
[19] Test Loss: 0.1270, Accuracy: 96.16%
[20] Test Loss: 0.1216, Accuracy: 96.15%
[21] Test Loss: 0.1174, Accuracy: 96.42%
[22] Test Loss: 0.1161, Accuracy: 96.40%
[23] Test Loss: 0.1123, Accuracy: 96.46%
[24] Test Loss: 0.1109, Accuracy: 96.57%
[25] Test Loss: 0.1105, A